In [ ]:
# Install all required packages
%package install bloomberg.bquant.signal_lab=1.5.2

In [ ]:
import bloomberg.bquant.signal_lab as signal_lab
signal_lab.__version__

In [1]:
import bql

from bloomberg.bquant.signal_lab.workflow.node import (
    industry_grouping, portfolio_construction)
from bloomberg.bquant.signal_lab.signal.transformers import WeightingScheme
from bloomberg.bquant.signal_lab.workflow.factory import (
    UniverseFactory,
    DataItemFactory,
    SignalFactory,
)
from bloomberg.bquant.signal_lab.workflow import (
    AnalyticsDataConfig,
    build_backtest,
)

from bloomberg.bquant.signal_lab.workflow.utils import get_sandbox_path

import utils.event_backtest_helper as ebh
import backtest_params as bp

import numpy as np
import pandas as pd
import importlib

In [2]:
importlib.reload(ebh)

<module 'utils.event_backtest_helper' from '/project/utils/event_backtest_helper.py'>

In [3]:
# Get the saved DataPack path
bq = bql.Service()
data_pack_path = f"{get_sandbox_path()}/esl/datapack_snapshot"
data_pack_path

's3://awmgd-prod-finml-sandbox-user/bclarke16/esl/datapack_snapshot'

In [4]:
# Key backtest parameters
start = "2020-01-05"
end = "2024-03-01"
universe_name = "INDU Index"

In [5]:
universe, benchmark, trading_calendar = bp.get_universe_params(start, end, universe_name, data_pack_path)
price, cur_mkt_cap, total_return = bp.get_return_params(start, end, data_pack_path)
analytics_data_config = bp.get_analytics_data_config(start, end, universe_name, data_pack_path)

In [6]:
# Import the test data set while developing
df = pd.read_csv('test_analysis.csv')

In [7]:
df = df[['Date','Security','Decision','Confidence']]

In [23]:
df = df.head(1)

In [24]:
trading_dates = list(df['Date'].unique())

In [25]:
def _bql_execute_single(univ: list[str], field: dict[str, bql.om.bql_item.BqlItem]) -> pd.DataFrame:
    """Execute a BQL query with one field"""
    req = bql.Request(univ, field)
    data = bq.execute(req)
    return data[0].df()

def convert_to_figi(df: pd.DataFrame) -> pd.DataFrame:
    """Function to convert Bloomberg tickers in a dataframe to FIGIs for ESL"""
    univ      = df['Security'].to_list()
    field     = {'figi': bq.data.composite_id_bb_global()}
    figi      = _bql_execute_single(univ, field)
    merged_df = df.merge(figi, left_on='Security', right_index=True).sort_index()
    return merged_df[['Date', 'figi', 'Decision', 'Confidence']].rename(columns={'figi':'Security'})
    

In [26]:
df1 = convert_to_figi(df)

In [27]:
df1

,Date,Security,Decision,Confidence
0,2020-02-06,BBG000BP52R2,BUY,70.0


In [11]:
def build_port_weights(signal: pd.DataFrame, events_df: pd.DataFrame) -> pd.DataFrame:
    long_portfolio =  signal.copy(deep=True)
    short_portfolio = signal.copy(deep=True)
    
    long_portfolio.loc[:,:] = False
    short_portfolio.loc[:,:] = False
    signal.loc[:,:] = 1
    
    # STEP 1 get the list of securities in th df_events database
    unique_securities = list(events_df['Security'].unique())
    
    # STEP 2: iterate over the list of securities to look at the individual trades
    for security in unique_securities:
        try:
            security_trades = events_df[events_df['Security'] == security]
        
            # STEP 3: iterate over the trades and update the long/ short portfolio depending on trade direction
            for row in security_trades.itertuples():
                if row.Decision == 'BUY':
                    long_portfolio[security].loc[row.Date:] = True
                    short_portfolio[security].loc[row.Date:] = False
                if row.Decision == 'SELL':
                    long_portfolio[security].loc[row.Date:] = False
                    short_portfolio[security].loc[row.Date:] = True
                if row.Decision == 'HOLD':
                    continue
                if row.Decision == 'Missing':
                    continue
        except KeyError:
            print(f"Missing: {security}")
    
        # STEP 4: create an equal weighted long and short leg
    long_portfolio_leg = ebh.leg_portfolio(
        signal=signal,
        weighting_scheme=WeightingScheme.EQUAL,
        assets_filter=long_portfolio,
        long_leg=True
    )
    
    short_portfolio_leg = ebh.leg_portfolio(
        signal=signal,
        weighting_scheme=WeightingScheme.EQUAL,
        assets_filter=short_portfolio,
        long_leg=False
    )
    
    long_short_portfolio = long_portfolio_leg.add(
        short_portfolio_leg,
        fill_value=0.0,
    )
    
    # STEP 4: return the long and short portfolios
    return long_short_portfolio#long_portfolio_leg, short_portfolio_leg

In [12]:
def event_backtest(df_events: pd.DataFrame, universe: UniverseFactory, signal: SignalFactory) -> pd.DataFrame:
    """Take a list of trades and create the weights in the portfolio
    df_events: DataFrame with Date, Security, Decision (BUY/ SELL/ HOLD) and Confidence Columns
    universe:  UniverseFactory object from Bloomberg Equity Signal Lab
    signal:    SignalFactory object from Bloomberg Equity Signal Lab - this is usually price
    """
    converted_df = convert_to_figi(df_events)
    
    signal.bind_universe(universe)
    signal_df = signal.df()
    return build_port_weights(signal_df,converted_df)

In [13]:
lsp = event_backtest(df, universe, price) 

100%|██████████| 1/1 [00:05<00:00,  5.33s/it]


Missing: BBG000BBJQV0
Missing: BBG00BN961G4
Missing: BBG000BSXQV7


In [14]:
price.bind_universe(universe)
price_df = price.df()

100%|██████████| 1/1 [00:04<00:00,  4.82s/it]


In [28]:


port_long_short = portfolio_construction.from_user(
    compute_weights_fn=build_port_weights,
    total_returns=total_return,
    trading_calendar=trading_calendar,
    implementation_lag=1,
    rebalance_freq="D",
    #trading_dates=trading_dates,
    events_df=df1,
    signal=price_df,
)

In [29]:
def price_signal(signal:  DataItemFactory) -> DataItemFactory:
    return signal

prices_signal = SignalFactory.from_user(
    user_func=price_signal,
    start=start,
    end=end,
    label="market_cap",
    signal=total_return,
)


backtest = build_backtest(
    universe=universe,                                  # Univ of choice from DataPack
    benchmark_universe=benchmark,                       # Benchmark of choice from DataPack 
    start=start,                                        # Backtest start date
    end=end,                                            # Backtest end date
    namespace='events-bt',                 # The user S3 sandbox storage 
    signals=[prices_signal],                                 # My list of signals to use

    portfolio_construction = port_long_short,

    reports=[
        "PerformanceReport",
        "QuantileAnalyticsReport",
        "DescriptiveStatisticsReport",
    ],
    analytics_data_config= analytics_data_config,
)

In [30]:
backtest.signals

{'market_cap': <bloomberg.bquant.compute_graph.model.node.Node at 0x7fe47f4bfe50>}

In [31]:
backtest_results = backtest.evaluate_graph()

100%|██████████| 117/117 [00:32<00:00,  3.61it/s]


In [32]:
#backtest_results

In [33]:
backtest_results.plot_analytics()

In [ ]:
sp

In [ ]:
long_portfolio = price.df()

In [ ]:
short_portfolio = long_portfolio.copy(deep=True)

In [ ]:
long_portfolio.loc[:,:] = False
short_portfolio.loc[:,:] = False

In [ ]:
df1

In [ ]:
df1[df1['Decision'] == 'BUY']

In [ ]:
unique_securities = list(df1['Security'].unique())
#unique_securities

In [ ]:
for security in unique_securities:
    print(security)
    security_trades = df1[df1['Security'] == security]

    # STEP 3: iterate over the trades and update the long/ short portfolio depending on trade direction
    for row in security_trades.itertuples():
        if row.Decision == 'BUY':
            long_portfolio[security].loc[row.Date:] = True
            short_portfolio[security].loc[row.Date:] = False
        if row.Decision == 'SELL':
            long_portfolio[security].loc[row.Date:] = False
            short_portfolio[security].loc[row.Date:] = True
        if row.Decision == 'HOLD':
            continue
        if row.Decision == 'Missing':
            continue

In [ ]:
universe.df()

In [ ]:
# modify the price signal
price.bind_universe(universe)
price_df = price.df()

In [ ]:
price_df.loc[:,:] = False

In [ ]:
secs = list(price_df.columns)

In [ ]:
req = bql.Request(secs, bq.data.name())
data = bq.execute(req)
data[0].df() # BBG000BP52R2

In [ ]:
price_df['BBG000BCQZS4'].loc['2024-02-28':] = False


In [ ]:
price_df